In [14]:
import pandas as pd
import os
import configparser
import itertools
import sqlite3
import time
import random
import locale
import logging
from logging.config import fileConfig
import traceback
from tools.utils import *

# 选择标注数据

In [62]:
data = pd.read_excel('data/full_讲题_10000_-1_0301_使用２.4pattern.xlsx')

In [71]:
data_p_index = data.段index.unique().tolist()

In [76]:
random.seed(2019)
final_index = random.sample(data_p_index[1:100], 50) + random.sample(
    data_p_index[100:2000], 150) + random.sample(data_p_index[2000:], 100)

In [80]:
def get_text(df):
    line_list = []
    for index,row in df.iterrows():
        line = '{}:{}'.format(row['who'],row['text'])
        line_list.append(line)
    return line_list

In [87]:
for index in final_index:
    df = data[data.段index==index]
    line_list = get_text(df)
    output_path = os.path.join(raw_data_dir,'{}.txt'.format(int(index)))
    with open(output_path,'w') as f:
        f.write('\n'.join(line_list))

# 标注数据准备

In [85]:
def save_pairs_2_db():
    name_files = os.listdir(raw_data_dir)
    name_files = [x.split('.')[0] for x in name_files]
    pair_list = list(itertools.combinations(name_files,2))
    random.seed(2019)
    random.shuffle(pair_list)
    if os.path.exists(db_path):
        logger.info('Remove old db,path is {}'.format(db_path))
        os.remove(db_path)
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    logger.info('Create table,name is {}'.format(table))

    # Create table
    c.execute('''CREATE TABLE {}
             (id,index_i, index_j,label,state,create_time,label_time,user)'''.
          format(table))
    logger.info('Write pairs to table {}'.format(table))
    for id, pair in enumerate(pair_list):
        create_time = getNowTime()
        c.execute("INSERT INTO pairs VALUES ({},{},{},{},{},'{}','{}','{}')".format(
        id + 1, pair[0], pair[1], -1, wait_state, create_time, '',''))
    conn.commit()
    conn.close()
    logger.info('Finish create table {}'.format(table))

In [86]:
save_pairs_2_db()

In [ ]:
%%time
test_pair = Pair()
test_pair.save(1)

In [ ]:
result = c.execute("select * from pairs where state == 0 limit 10")

In [ ]:
c.execute('UPDATE {} SET label =  WHERE id = {}'.format(table,1,1))
result = conn.

In [ ]:
# result = c.execute("select * from pairs where label= -1 and state==0 limit 10")
result = c.execute('UPDATE {} SET label ={}  WHERE id = {}'.format(table,1,1))
result = [dict(x) for x in result]
pd.DataFrame(result)

In [11]:
# test

In [12]:
import dash_html_components as html